In [5]:
global scored
import math
import pandas as pd
import numpy as np
df = pd.read_csv(r'players_21.csv')

In [6]:
#Deleting unnecessary columns
delete = ["player_url","sofifa_id","long_name","age","dob","height_cm","weight_kg","league_rank","team_position","nationality","value_eur","wage_eur","international_reputation","body_type","real_face","release_clause_eur","team_jersey_number","loaned_from","joined","contract_valid_until","nation_position","nation_jersey_number","defending_marking","goalkeeping_diving","goalkeeping_handling","goalkeeping_kicking","goalkeeping_positioning","goalkeeping_reflexes","potential","ls","rs","lw","lf","st","cf","rf","rw","lam","cam","ram","lm","lcm","cm","rcm","rm","lwb","ldm","cdm","rdm","rwb","lb","lcb","cb","rcb","rb"]
df.drop(columns = delete, inplace = True)
df["defending_marking"] = df["defending_standing_tackle"]

#Grouping leagues and club names
grouped = df.groupby(df["league_name"])
clubs = grouped.get_group("English Premier League")
epl = (clubs["club_name"].value_counts())
epl = dict(epl)
epl = (list(epl.keys()))
epl = sorted(epl)
final_table = []

grouped = df.groupby(df["club_name"])
club = df["club_name"].value_counts()
league = df["league_name"].value_counts()
league = dict(league)
league_n = list(league.keys())
league_n = sorted(league_n)
club = dict(club)
club_n = list(club.values())
club_k = list(club.keys())
tabled = []

In [7]:
#simulating an entire season (english premier league)
for england in range(len(epl)):
    club_na = 0
    for eng in range(len(epl)):
        if england == eng:
            continue
        inputs = epl[england]
        inputs2 = epl[eng]
        for i in range(len(club_k)):
            if inputs == club_k[i]:
                club1 = grouped.get_group(club_k[i])
            if inputs2 == club_k[i]:
                club2 = grouped.get_group(club_k[i])
        
        club1 = club1.sort_values('overall',ascending=False)
        club1.reset_index(drop=True, inplace=True)
        club2 = club2.sort_values('overall',ascending=False)
        club2.reset_index(drop=True, inplace=True)
        
        overall = list(club1["overall"])
        overall2 = list(club2["overall"])
        pos = list(club1["player_positions"])
        pos2 = list(club2["player_positions"])
        
        def best_eleven(pos):
            keep = []
            de = []
            st = 0
            mid = 0
            deb = 0
            gk = 0
            for i in range (len(pos)):
                if (st < 3) and ('ST' in pos[i] or 'RW' in pos[i] or 'LW' in pos[i] or 'CF' in pos[i]):
                    keep.append("fw")
                    st = st + 1
                elif (gk == 0) and ('GK' in pos[i]):
                    keep.append("gk")
                    gk = 1
                elif (mid < 3) and ('RM' in pos[i] or 'LM' in pos[i] or 'CM' in pos[i] or 'CAM' in pos[i] or 'CDM' in pos[i]):
                    keep.append("mid")
                    mid = mid + 1
                elif (deb < 4) and ('CB' in pos[i] or 'RB' in pos[i] or 'LB' in pos[i] or 'LWB' in pos[i] or 'RWB' in pos[i]):
                    keep.append("def")
                    deb = deb + 1
                else:
                    de.append(i)
            return de, keep
        
        def mean(x):
            return sum(list(x))/len(x)
        
        def kickoff(x,y):
            overall = (mean(x["overall"]) + mean(x["pace"]))/2
            overall2 = (mean(y["overall"]) + mean(y["pace"]))/2
            if overall >= overall2:
                return 1
            else:
                return 0
        
        
        de,keep = best_eleven(pos)
        de2,keep2 = best_eleven(pos2)
        
        club1 = club1.drop(labels=de, axis=0)
        club1.reset_index(drop=True, inplace=True)
        club2 = club2.drop(labels=de2, axis=0)
        club2.reset_index(drop=True, inplace=True)
        club1["player_pos"] = keep
        club2["player_pos"] = keep2
        del club1["player_positions"]
        del club2["player_positions"]
        grouped1 = club1.groupby(club1["player_pos"])
        grouped2 = club2.groupby(club2["player_pos"])
        mf1 = grouped1.get_group("mid")
        mf1.reset_index(drop=True, inplace=True)
        mf2 = grouped2.get_group("mid")
        mf2.reset_index(drop=True, inplace=True)
        fw1 = grouped1.get_group("fw")
        fw1.reset_index(drop=True, inplace=True)
        fw2 = grouped2.get_group("fw")
        fw2.reset_index(drop=True, inplace=True)
        def1 = grouped1.get_group("def")
        def1.reset_index(drop=True, inplace=True)
        def2 = grouped2.get_group("def")
        def2.reset_index(drop=True, inplace=True)
        gk1 = grouped1.get_group("gk")
        gk1.reset_index(drop=True, inplace=True)
        gk2 = grouped2.get_group("gk")
        gk2.reset_index(drop=True, inplace=True)    
        
        def probs (i,c):
            if c == 1:
                x = np.random.choice(np.arange(0,2), p=[0.5-i,0.5+i])
                return x
            else:
                x = np.random.choice(np.arange(0,2), p=[0.5+i,0.5-i])
                return x
        def midfield(x,x1,x2,gk1,y,y1,y2,gk2):
            i = float(mean(x["passing"]) - mean(y2["defending"]))
            if i > 0:
                i = i/100
                k = probs(i,1)
            else:
                i = abs(i)/100
                k = probs(i,0)
            if k:
                passing = (mean(x["attacking_short_passing"]) + mean(x["skill_long_passing"]))/2
                tackling = (mean(y2["defending_marking"]) + mean(y2["mentality_interceptions"]))/2
                diff = float(passing - tackling)
                if diff > 0:
                    u = probs(diff/100,1)
                else:
                    u = probs(abs(diff)/100,0)
                if u:
                    what = attack(x,x1,x2,gk1,y,y1,y2,gk2)
                    if what:
                        return 1
            else:
                return 0
        def attack(x,x1,x2,gk1,y,y1,y2,gk2):
            i = float(mean(x1["dribbling"]) - mean(y2["physic"]))
            if i > 0:
                i = i/100
                k = probs(i,1)
                main = []
                if k:
                    att = (mean(x1["skill_dribbling"]) + mean(x1["skill_ball_control"]))/2
                    defend = (mean(y2["defending_standing_tackle"]) + mean(y2["movement_reactions"]) + mean(y2["power_strength"]))/3
                    u = float(att - defend)
                    if u > 0:
                        main.append(probs(u/100,1))
                    else:
                        main.append(probs(abs(u)/100,0))
                    att = (mean(x1["movement_acceleration"]) + mean(x1["movement_sprint_speed"]) + mean(x1["movement_agility"]) + mean(x1["movement_reactions"]) + mean(x1["movement_balance"]))/5
                    defend = (mean(y2["power_strength"]) + mean(y2["defending_sliding_tackle"]))/2
                    u = float(att - defend)
                    if u > 0:
                        main.append(probs(u/100,1))
                    else:
                        main.append(probs(abs(u)/100,0))
                    att = (mean(x1["attacking_crossing"]) + mean(x1["attacking_heading_accuracy"]))/2
                    defend = (mean(y2["defending_marking"]) + mean(y2["power_jumping"]))/2
                    u = float(att - defend)
                    if u > 0:
                        main.append(probs(u/100,1))
                    else:
                        main.append(probs(abs(u)/100,0))
                    if sum(main) >= 2:
                        return score(x1,gk2)
            else:
                return 0
        
        def score(x,y):
            main = []
            att = mean(x["power_shot_power"]) 
            gk = (y["gk_positioning"] + y["gk_diving"])/2
            f = float(att - gk)
            if f > 0:
                main.append(probs(f/100,1))
            else:
                main.append(probs(abs(f)/100,0))
            att = mean(x["power_long_shots"]) 
            gk = y["gk_positioning"]
            f = float(att - gk)
            if f > 0:
                main.append(probs(f/100,1))
            else:
                main.append(probs(abs(f)/100,0))
            att = mean(x["attacking_volleys"])
            gk = (y["gk_reflexes"] + y["gk_speed"])/2
            f = float(att - gk)
            if f > 0:
                main.append(probs(f/100,1))
            else:
                main.append(probs(abs(f)/100,0))
            att = mean(x["skill_curve"])
            gk = (y["gk_positioning"] + y["gk_diving"])/2
            f = float(att - gk)
            if f > 0:
                main.append(probs(f/100,1))
            else:
                main.append(probs(abs(f)/100,0))
            att = mean(x["skill_fk_accuracy"])
            gk = (y["gk_reflexes"] + y["gk_handling"])/2
            f = float(att - gk)
            if f > 0:
                main.append(probs(f/100,1))
            else:
                main.append(probs(abs(f)/100,0))
            hit = 0
            miss = 0
            for index in range(len(main)):
                if main[index] == 1:
                    hit = hit + 1
                else:
                    miss = miss + 1
            if hit > miss:
                att = mean(x["attacking_finishing"])
                gk = (y["gk_diving"] + y["gk_handling"] + y["gk_kicking"] + y["gk_reflexes"] + y["gk_speed"] + y["gk_positioning"])/6
                f = float(att - gk)
                if f > 0:
                    return probs(f/100,1)
                else:
                    return probs(abs(f)/100,0)
            else:
                return 0
            
        team1 = 0
        team2 = 0
        over = 30
        home = 70
        away = 30
        if 0:
            if kickoff(mf1,mf2):
                for i in range(home+over):
                    k = midfield(mf1,fw1,def1,gk1,mf2,fw2,def2,gk2)
                    if k:
                        team1 = team1 + 1
                for i in range(away+over-15):
                    k = midfield(mf2,fw2,def2,gk2,mf1,fw1,def1,gk1)
                    if k:
                        team2 = team2 + 1
            else:
                for i in range(home+over-10):
                    k = midfield(mf1,fw1,def1,gk1,mf2,fw2,def2,gk2)
                    if k:
                        team1 = team1 + 1
                for i in range(away+over):
                    k = midfield(mf2,fw2,def2,gk2,mf1,fw1,def1,gk1)
                    if k:
                        team2 = team2 + 1
            final_table.append(inputs + ' ' + str(team1) + " - " + str(team2) + ' ' + inputs2)
            if team1 > team2:
                club_na = club_na + 3
            if team1 == team2:
                club_na = club_na + 1            
        else:
            if kickoff(mf2,mf1):
                for i in range(home+over):
                    k = midfield(mf2,fw2,def2,gk2,mf1,fw1,def1,gk1)
                    if k:
                        team2 = team2 + 1
                for i in range(away+over-15):
                    k = midfield(mf1,fw1,def1,gk1,mf2,fw2,def2,gk2)
                    if k:
                        team1 = team1 + 1
            else:
                for i in range(home+over-10):
                    k = midfield(mf2,fw2,def2,gk2,mf1,fw1,def1,gk1)
                    if k:
                        team2 = team2 + 1
                for i in range(away+over):
                    k = midfield(mf1,fw1,def1,gk1,mf2,fw2,def2,gk2)
                    if k:
                        team1 = team1 + 1
            final_table.append(inputs2 + ' ' + str(team2) + " - " + str(team1) + ' ' + inputs)
            if team1 > team2:
                club_na = club_na + 3
            if team1 == team2:
                club_na = club_na + 1            
    tabled.append(club_na)

In [8]:
srno = []
for i in range(1, 21):
    srno.append(i)
fin = { 'Club': epl, 'Points': tabled}
fin = pd.DataFrame(fin)
fin = fin.sort_values('Points',ascending = False)
fin.reset_index(drop=True, inplace=True)
fin["Points"] = fin["Points"]*2
fin["Position"] = srno
first_column = fin.pop('Position')
fin.insert(0, 'Position', first_column)
print(fin)
#fin = fin.to_html()

    Position                     Club  Points
0          1          Manchester City     104
1          2                Liverpool      94
2          3                  Arsenal      84
3          4        Tottenham Hotspur      76
4          5        Manchester United      68
5          6                  Chelsea      68
6          7           Leicester City      56
7          8  Wolverhampton Wanderers      40
8          9              Southampton      36
9         10   Brighton & Hove Albion      34
10        11           Crystal Palace      32
11        12                  Everton      30
12        13          West Ham United      30
13        14     West Bromwich Albion      24
14        15                  Burnley      24
15        16              Aston Villa      22
16        17         Newcastle United      22
17        18                   Fulham      16
18        19             Leeds United      14
19        20         Sheffield United       4
